In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Echarts Jupyter lab support
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

In [ ]:
# Echarts imports
from pyecharts import options as opts
from pyecharts.globals import  ThemeType
from pyecharts.charts import Line

In [ ]:
from IPython.display import HTML
import matplotlib.pyplot as plt

In [ ]:
from common.log_helper import LogHelper
from backtest.scale_backtest_args import ScaleBacktestArgGen
from backtest.scale_chain_reader import ScaleChainReader
from notebooks.img_symbols import down_arrow
from notebooks.echart_df_plot import plot_df_simple
from modelservice.scale_alpha import ScaleAlpha
from modelservice.alpha_consts import SignalDirection
import logging
import sys
import pandas as pd
import talib

In [ ]:
LogHelper.configure_logging(verbose=False)
pd.set_option("display.precision", 2)
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
ReaderArgs = {
    'common': {
        'start_time': '09:15:00',
        'end_time' : '15:25:00',
    },
}
AlphaArgs = {
    'sigmult' : 0.5,
    'smoother' : {
        'TS' : {'enabled': False , 'type' : 'STABLEWIN' , 'timeperiod' : 3 * 60 },
        'UL' : {'enabled': True, 'type' : 'EMA' , 'timeperiod' : 3 * 60 },
        'CP' : {'enabled': False, 'type' : 'AGG' , 'timeperiod' : 3 * 60 },            
    }
}

In [ ]:
dump_base_dir = "/home/qoptrprod/data/dumps/"
trading_date = "20240319"
underlying = "MIDCPNIFTY_I"

In [ ]:
arg_obj = ScaleBacktestArgGen.resolve_and_gen_args(dump_base_dir, underlying, trading_date, ReaderArgs)
reader = ScaleChainReader(arg_obj)
alpha = ScaleAlpha(AlphaArgs)

In [ ]:
show_distict_signal = False

color_none = 'rgba(100, 100, 100, 0.5)'
color_red = 'rgba(230,50,120,0.8)' 
color_green = 'rgba(50,220,120,0.8)'

valid_signals = []
signal_mark_points = []
last_signal_dir = None
for chain in reader.read_chain():
    signal = alpha.process_update(chain)
    #-- Filter --
    if not signal.is_valid:
        continue

    valid_signals.append(signal)    

    #--- Add Signal Markers ---
    if signal.dir == SignalDirection.BUY or signal.dir == SignalDirection.SELL:
        if show_distict_signal:
            if last_signal_dir == signal.dir:
                continue
            last_signal_dir = signal.dir   
        color = color_green
        if signal.dir == SignalDirection.BUY:
            color = color_green
        elif signal.dir == SignalDirection.SELL:
            color = color_red
        signal_mark_points.append(opts.MarkPointItem(coord=[signal.ulexts, signal.ulpx], 
                symbol='circle',               
                symbol_size=[8, 8],
                itemstyle_opts=opts.ItemStyleOpts(color=color),
                ))

print(valid_signals[0])
df = pd.DataFrame(valid_signals)
_ = df.set_index('ulexts')

In [ ]:
plot1 = plot_df_simple(
    df, 
    x_col = 'ulexts', 
    y_cols = ['ulpx' , 'smooth_ulpx', 'atm'], 
    mark_points = {'ulpx' :  signal_mark_points},
    title= f"Signal Analysis - {underlying} {trading_date}"
)

In [ ]:
plot1.render_notebook()

In [ ]:
plot2 = plot_df_simple(
    df, 
    x_col = 'ulexts', 
    y_cols = ['cpx' , 'ppx'], 
    mark_points = {},
    title= f"Call Put Prices"
)

In [ ]:
plot2.render_notebook()

In [ ]:
plot3 = plot_df_simple(
    df, 
    x_col = 'ulexts', 
    y_cols = ['val'], 
    mark_points = {},
    title= f"Signal Values"
)

In [ ]:
plot3.render_notebook()

In [ ]:
import seaborn as sns
import statsmodels
import numpy as np

In [ ]:
def corr_plot(df, shift, idx, axs):
    df['emapx'] = talib.EMA(df['ulpx'], timeperiod=10*60)
    df.dropna()
    #df['sulpx'] = df['smooth_ulpx'].shift(-shift)
    df['sulpx'] = df['emapx'].shift(-shift)
    #df['diff'] = df['sulpx'] - df['smooth_ulpx']
    df['diff'] = df['sulpx'] - df['emapx']
    #df['diff'] = df['diff'] / df['diff'].abs()
    ddf = df
    ddf = df.query('val > 12.5 or val < -12.5')
    #display(ddf.head(11))
    sns.regplot(data=ddf, x='val', y='diff', ax=axs[idx], scatter=True)
    axs[idx].set_ylabel(f"shifted:{shift} sec", labelpad=25, fontsize=12)
    corr = ddf['diff'].corr(ddf['val'])
    print(f"shift:{shift} val=>diff corr = {corr}")

In [ ]:
shifts = [1, 1*60, 2*60, 3*60, 5*60, 30*60]
fig, axs = plt.subplots(nrows=len(shifts), figsize=(7, 12))
_ = [corr_plot(df, shift=shift, idx=idx, axs=axs) for idx, shift in enumerate(shifts)]